In [1]:
import pandas as pd
import os
import re
from io import StringIO

In [2]:
df = pd.read_csv("labels/Anno_coarse/list_bbox.txt", delim_whitespace=True, header=1)
df.head()

C:\Users\elise\AppData\Local\Temp\ipykernel_21384\1090048282.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("labels/Anno_coarse/list_bbox.txt", delim_whitespace=True, header=1)


,image_name,x_1,y_1,x_2,y_2
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,72,79,232,273
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,67,59,155,161
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,65,65,156,200
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,51,62,167,182
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,46,88,166,262


In [3]:
file_path = 'labels/Anno_coarse/list_attr_cloth.txt'

transformed_lines = []
with open(file_path, 'r') as file:
    for line in file:
        transformed_line = re.sub(r' (?=\S)(?!\d)', '_', line.strip())
        transformed_line = transformed_line.title()
        transformed_lines.append(transformed_line)

data = '\n'.join(transformed_lines)
data = StringIO(data)

raw_attributes = pd.read_csv(data, sep='\s+', header=1)
column_names = ['attribute_name', 'attribute_type']
raw_attributes.columns = column_names
raw_attributes.head(10)

<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:13: SyntaxWarning: invalid escape sequence '\s'
C:\Users\elise\AppData\Local\Temp\ipykernel_21384\2376651624.py:13: SyntaxWarning: invalid escape sequence '\s'
  raw_attributes = pd.read_csv(data, sep='\s+', header=1)


,attribute_name,attribute_type
0,A-Line,3
1,Abstract,1
2,Abstract_Chevron,1
3,Abstract_Chevron_Print,1
4,Abstract_Diamond,1
5,Abstract_Floral,1
6,Abstract_Floral_Print,1
7,Abstract_Geo,1
8,Abstract_Geo_Print,1
9,Abstract_Paisley,1


In [4]:
count_values = []
for row in raw_attributes.itertuples():
    attribute_name = getattr(row, 'attribute_name')
    attribute_type = getattr(row, 'attribute_type')
    count = df['image_name'].str.contains(attribute_name, regex=True).sum()  # Counts the number of True values
    count_values.append([attribute_name, attribute_type, count])  # Store texture and its count

count_df = pd.DataFrame(count_values, columns=['attribute_name', 'attribute_type', 'count'])
count_df

,attribute_name,attribute_type,count
0,A-Line,3,3252
1,Abstract,1,10496
2,Abstract_Chevron,1,231
3,Abstract_Chevron_Print,1,110
4,Abstract_Diamond,1,151
...,...,...,...
995,Zip-Pocket,4,139
996,Zip-Up,4,291
997,Zipped,4,83
998,Zipper,4,3465


In [5]:
selected_rows = []  # A list to hold the rows that meet the condition
for row in count_df.itertuples():
    if row.count > 1000:
        selected_rows.append(row)  # Append the row as a dictionary

attribute = pd.DataFrame(selected_rows)
attribute.drop(columns=['Index'], inplace=True)
attribute

,attribute_name,attribute_type,count
0,A-Line,3,3252
1,Abstract,1,10496
2,Abstract_Print,1,4239
3,Acid,2,1442
4,Acid_Wash,2,1415
...,...,...,...
187,Workout,5,1130
188,Woven,2,5687
189,Zip,4,4996
190,Zipper,4,3465


In [6]:
texture = attribute[attribute['attribute_type'] == 1]
fabric = attribute[attribute['attribute_type'] == 2]
shape = attribute[attribute['attribute_type'] == 3]
part = attribute[attribute['attribute_type'] == 4]
style = attribute[attribute['attribute_type'] == 5]

In [7]:
texture = texture.reset_index(drop=True) # Reset the index
texture = texture.reset_index(drop=False) # Display the index
texture = texture.drop(columns=['count','attribute_type'])
texture.head()

,index,attribute_name
0,0,Abstract
1,1,Abstract_Print
2,2,Baroque
3,3,Chevron
4,4,Colorblock


In [8]:
fabric = fabric.reset_index(drop=True) # Reset the index
fabric = fabric.reset_index(drop=False) # Display the index
fabric = fabric.drop(columns=['count','attribute_type'])
fabric.head()

,index,attribute_name
0,0,Acid
1,1,Acid_Wash
2,2,Bead
3,3,Beaded
4,4,Burnout


In [9]:
shape = shape.reset_index(drop=True) # Reset the index
shape = shape.reset_index(drop=False) # Display the index
shape = shape.drop(columns=['count','attribute_type'])
shape.head()

,index,attribute_name
0,0,A-Line
1,1,Asymmetric
2,2,Bodycon
3,3,Box
4,4,Boxy


In [10]:
part = part.reset_index(drop=True) # Reset the index
part = part.reset_index(drop=False) # Display the index
part = part.drop(columns=['count','attribute_type'])
part.head()

,index,attribute_name
0,0,Belted
1,1,Bow
2,2,Button
3,3,Buttoned
4,4,Collar


In [11]:
style = style.reset_index(drop=True) # Reset the index
style = style.reset_index(drop=False) # Display the index
style = style.drop(columns=['count','attribute_type']) 
style.head()

,index,attribute_name
0,0,Athletic
1,1,Babydoll
2,2,Baseball
3,3,Boyfriend
4,4,Classic


In [17]:
def creer_fichier(chemin_fichier):
    '''Créer les dossier et fichiers nécessaires pour le fichier txt ou les images
    chemin_fichier : str'''
    dossier = os.path.dirname(chemin_fichier)
    if not os.path.exists(dossier):
        os.makedirs(dossier)

In [21]:
def creer_csv(content, chemin_fichier, new_column_names=None):
    creer_fichier(chemin_fichier)
    if new_column_names:
        content = content.rename(columns=new_column_names)
    content.to_csv(chemin_fichier, index=False)

In [39]:
path = 'labels/database_csv/'
creer_csv(texture, path + 'texture.csv', {'index': 'id_texture', 'attribute_name': 'texture'})
creer_csv(fabric, path + 'fabric.csv', {'index': 'id_fabric', 'attribute_name': 'fabric'})
creer_csv(shape, path +'shape.csv', {'index': 'id_shape', 'attribute_name': 'shape'})
creer_csv(part, path +'part.csv', {'index': 'id_part', 'attribute_name': 'part'})
creer_csv(style, path +'style.csv', {'index': 'id_style', 'attribute_name': 'style'})

In [90]:
category = pd.read_csv('labels/Anno_coarse/categories.txt', header=0)
category.head()

,category_name,category_type
0,Anorak,0
1,Blazer,1
2,Blouse,2
3,Bomber,3
4,Button-Down,4


In [91]:
def inverser_colonnes(df, col1, col2):
    # Copie de la liste des noms de colonnes pour éviter de modifier l'original
    cols = df.columns.tolist()
    # Obtenir les indices des colonnes à échanger
    idx1, idx2 = cols.index(col1), cols.index(col2)
    # Échanger les noms des colonnes
    cols[idx1], cols[idx2] = cols[idx2], cols[idx1]
    # Réaffecter l'ordre modifié au DataFrame
    df = df[cols]
    return df

In [92]:
category = inverser_colonnes(category, 'category_name', 'category_type')
category.head()

,category_type,category_name
0,0,Anorak
1,1,Blazer
2,2,Blouse
3,3,Bomber
4,4,Button-Down


In [36]:
creer_csv(category, 'labels/database_csv/category.csv', {'category_type': 'id_category', 'category_name': 'category'})

In [37]:
marque = pd.DataFrame({'id_brand': [0, 1, 2, 3, 4 , 5, 6, 7, 8, 9], 'brand': ['Zara', 'H&M', 'Forever 21', 'Asos', 'Jennifer', 'Promod', 'Uniqlo', 'Pull&Bear', 'ESPRIT', 'Primark']})
creer_csv(marque, 'labels/database_csv/brand.csv')

In [38]:
size = pd.DataFrame({'id_size': [0, 1, 2, 3, 4, 5, 6], 'size': ['XXS', 'XS', 'S', 'M', 'L', 'XL', 'XXL']})
creer_csv(size, 'labels/database_csv/size.csv')

In [106]:
df = pd.read_csv("labels/Anno_coarse/list_bbox.txt", delim_whitespace=True, header=1)

C:\Users\elise\AppData\Local\Temp\ipykernel_21384\166948314.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("labels/Anno_coarse/list_bbox.txt", delim_whitespace=True, header=1)


In [107]:
df = df.drop(columns=['x_1', 'y_1', 'x_2', 'y_2'])
df.rename(columns={'image_name': 'absolute_path'}, inplace=True)
df.head()

,image_name
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg


In [108]:
for row in df.itertuples():
    absolute_path = getattr(row, 'absolute_path')
    absolute_path = absolute_path.split('/')[1:-1]
    absolute_path = '/'.join(absolute_path)
    df.at[row.Index, 'absolute_path'] = absolute_path
    image_name = absolute_path.split('/')[1]
    df.at[row.Index, 'image_name'] = image_name
df.head()

,image_name
0,Sheer_Pleated-Front_Blouse
1,Sheer_Pleated-Front_Blouse
2,Sheer_Pleated-Front_Blouse
3,Sheer_Pleated-Front_Blouse
4,Sheer_Pleated-Front_Blouse


In [110]:
for row in df.itertuples():
    attributes = row.image_name.split('_')
    df.at[row.Index, 'attributes'] = attributes
df.head()

,image_name,attributes
0,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]"
1,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]"
2,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]"
3,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]"
4,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]"


In [111]:
for row in df.itertuples():
    for attribute in row.attributes:
        if not category[category['category_name'].str.contains(str(attribute), regex=True)].empty :
            num = category[category['category_name'].str.contains(str(attribute), regex=True)].index[0]
            df.at[row.Index, 'category'] = num
        if not texture[texture['attribute_name'].str.contains(str(attribute), regex=True)].empty :
            num = texture[texture['attribute_name'].str.contains(str(attribute), regex=True)].index[0]
            df.at[row.Index, 'texture'] = num
        if not fabric[fabric['attribute_name'].str.contains(str(attribute), regex=True)].empty :
            num = fabric[fabric['attribute_name'].str.contains(str(attribute), regex=True)].index[0]
            df.at[row.Index, 'fabric'] = num
        if not shape[shape['attribute_name'].str.contains(str(attribute), regex=True)].empty :
            num = shape[shape['attribute_name'].str.contains(str(attribute), regex=True)].index[0]
            df.at[row.Index, 'shape'] = num
        if not part[part['attribute_name'].str.contains(str(attribute), regex=True)].empty :
            num = part[part['attribute_name'].str.contains(str(attribute), regex=True)].index[0]
            df.at[row.Index, 'part'] = num
        if not style[style['attribute_name'].str.contains(str(attribute), regex=True)].empty :
            num = style[style['attribute_name'].str.contains(str(attribute), regex=True)].index[0]
            df.at[row.Index, 'style'] = num
df.head(10)


,image_name,attributes,fabric,category,texture,part,style,shape
0,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]",54.0,2.0,NaN,NaN,NaN,NaN
1,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]",54.0,2.0,NaN,NaN,NaN,NaN
2,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]",54.0,2.0,NaN,NaN,NaN,NaN
3,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]",54.0,2.0,NaN,NaN,NaN,NaN
4,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]",54.0,2.0,NaN,NaN,NaN,NaN
5,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]",54.0,2.0,NaN,NaN,NaN,NaN
6,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]",54.0,2.0,NaN,NaN,NaN,NaN
7,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]",54.0,2.0,NaN,NaN,NaN,NaN
8,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]",54.0,2.0,NaN,NaN,NaN,NaN
9,Sheer_Pleated-Front_Blouse,"[Sheer, Pleated-Front, Blouse]",54.0,2.0,NaN,NaN,NaN,NaN


In [113]:
df.drop(columns=['attributes'], inplace=True)
df.drop(columns=['image_name'], inplace=True)
df['id_clothes'] = range(0, len(df))  # Starts the ID from 1 to the length of DataFrame
df.head()

,fabric,category,texture,part,style,shape,id_clothes
0,54.0,2.0,NaN,NaN,NaN,NaN,0
1,54.0,2.0,NaN,NaN,NaN,NaN,1
2,54.0,2.0,NaN,NaN,NaN,NaN,2
3,54.0,2.0,NaN,NaN,NaN,NaN,3
4,54.0,2.0,NaN,NaN,NaN,NaN,4


In [114]:
cols = df.columns.tolist()
# Obtenir les indices des colonnes à échanger
idx1, idx2, idx3, idx4, idx5, idx6, idx7 = cols.index('fabric'), cols.index('category'), cols.index('texture'), cols.index('part'), cols.index('style'), cols.index('shape'), cols.index('id_clothes')
# Échanger les noms des colonnes
cols[idx1], cols[idx2], cols[idx3], cols[idx4], cols[idx5], cols[idx6], cols[idx7] = cols[idx7], cols[idx2], cols[idx3], cols[idx1], cols[idx6], cols[idx4], cols[idx5]
# Réaffecter l'ordre modifié au DataFrame
df = df[cols]

In [115]:
df.head()

,id_clothes,category,texture,fabric,shape,part,style
0,0,2.0,NaN,54.0,NaN,NaN,NaN
1,1,2.0,NaN,54.0,NaN,NaN,NaN
2,2,2.0,NaN,54.0,NaN,NaN,NaN
3,3,2.0,NaN,54.0,NaN,NaN,NaN
4,4,2.0,NaN,54.0,NaN,NaN,NaN


In [117]:
creer_csv(df, 'labels/database_csv/clothes.csv', {'id_clothes': 'id_clothes', 'category': 'id_category', 'texture': 'id_texture', 'fabric': 'id_fabric', 'shape': 'id_shape', 'part': 'id_part', 'style': 'id_style'})

In [119]:
import random
for row in df.itertuples():
    brand = random.randint(0, 9)
    size = random.randint(0, 6)
    df.at[row.Index, 'brand'] = int(brand)
    df.at[row.Index, 'size'] = int(size)
df.head()

,id_clothes,category,texture,fabric,shape,part,style,brand,size
0,0,2.0,NaN,54.0,NaN,NaN,NaN,8.0,3.0
1,1,2.0,NaN,54.0,NaN,NaN,NaN,1.0,1.0
2,2,2.0,NaN,54.0,NaN,NaN,NaN,2.0,5.0
3,3,2.0,NaN,54.0,NaN,NaN,NaN,9.0,6.0
4,4,2.0,NaN,54.0,NaN,NaN,NaN,8.0,0.0


In [124]:
df['category'] = df['category'].astype('int8')
df['texture'] = df['texture'].astype('int8')
df['fabric'] = df['fabric'].astype('int8')
df['shape'] = df['shape'].astype('int8')
df['part'] = df['part'].astype('int8')
df['style'] = df['style'].astype('int8')
df['brand'] = df['brand'].astype('int8')
df['size'] = df['size'].astype('int8')
df.head()

C:\Users\elise\AppData\Local\Temp\ipykernel_21384\676560153.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = df['category'].astype('Int8')
C:\Users\elise\AppData\Local\Temp\ipykernel_21384\676560153.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['texture'] = df['texture'].astype('Int8')
C:\Users\elise\AppData\Local\Temp\ipykernel_21384\676560153.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

,id_clothes,category,texture,fabric,shape,part,style,brand,size
0,0,2,<NA>,54,<NA>,<NA>,<NA>,8,3
1,1,2,<NA>,54,<NA>,<NA>,<NA>,1,1
2,2,2,<NA>,54,<NA>,<NA>,<NA>,2,5
3,3,2,<NA>,54,<NA>,<NA>,<NA>,9,6
4,4,2,<NA>,54,<NA>,<NA>,<NA>,8,0


In [125]:
creer_csv(df, 'labels/database_csv/clothes.csv', {'id_clothes': 'id_clothes', 'category': 'id_category', 'texture': 'id_texture', 'fabric': 'id_fabric', 'shape': 'id_shape', 'part': 'id_part', 'style': 'id_style', 'brand': 'id_brand', 'size': 'id_size'})